<a href="https://colab.research.google.com/github/UznetDev/Kabr-prediction/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.linear_model import *
from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
import warnings
# warnings.filterwarnings('ignore')

In [ ]:
def feature_funk(df):
    df['Surface Area'] = 2 * (df['Length'] * df['Diameter'] + df['Length'] * df['Height'] + df['Diameter'] * df['Height'])
    df['Vol'] = df['Length'] * df['Diameter'] * df['Height']
    df['Vol_Hgt'] = (4/3) * np.pi * (df['Length']/2) * (df['Diameter']/2) * (df['Height']/2)

    df['Log_Wt'] = df['Weight'].agg(lambda x: np.log(x + 1) if x > 0 else np.nan)
    df['Length Bins'] = pd.qcut(df['Length'], q=4, labels=False)

    df['Wt_Red_Aft_Shk'] = df['Weight'] - df['Shucked Weight']
    df['Vis_Wt_Prop'] = df['Viscera Weight'] / df['Weight']

    df['Len_to_Dia_Ratio'] = df['Length'] / df['Diameter']
    df['Hgt_to_Wt_Ratio'] = df['Height'] / df['Weight']

    sex = {'I': 1, 'F': 3, 'M': 2}
    df['Sex'] = df['Sex'].replace(sex)
    df = df[df['Sex'].isin([1, 2, 3])]
    df.loc[:, 'Sex'] = pd.to_numeric(df['Sex'], errors='coerce')

    return df

train_df = feature_funk(pd.read_csv('train.csv'))


X = train_df.drop(columns=['id', 'Age'])
y = train_df['Age']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('poly', PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)),
            ('scaler', StandardScaler())
        ]), X.columns),
    ])
cv = KFold(n_splits=5, shuffle=True, random_state=42)



pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', HuberRegressor(alpha=0.0001, max_iter=100000))

])

cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring='neg_mean_absolute_error')

pipeline.fit(X, y)

<ipython-input-4-0884ef7e8b6a>:6: FutureWarning: using <function feature_funk.<locals>.<lambda> at 0x7f4d739cb9a0> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df['Log_Wt'] = df['Weight'].agg(lambda x: np.log(x + 1) if x > 0 else np.nan)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('poly',
                                                                   PolynomialFeatures(include_bias=False,
                                                                                      interaction_only=True)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Surface Area', 'Vol', 'Vol_Hgt',
       'Log_Wt', 'Length Bins', 'Wt_Red_Aft_Shk', 'Vis_Wt_Prop',
       'Len_to_Dia_Ratio', 'Hgt_to_Wt_Ratio'],
      dtype='object'))])),
                ('model', HuberRegressor(max_iter=100000))])

In [ ]:
test_df = feature_funk(pd.read_csv('test.csv'))
test_y = test_df.drop(columns=['id'])
y_pred = pipeline.predict(test_y)
seple_df = pd.DataFrame({'id': test_df['id'], 'Age': y_pred})
seple_df.to_csv('StandardScaler.csv', index=False)

<ipython-input-4-0884ef7e8b6a>:6: FutureWarning: using <function feature_funk.<locals>.<lambda> at 0x7f4d6350e0e0> in Series.agg cannot aggregate and has been deprecated. Use Series.transform to keep behavior unchanged.
  df['Log_Wt'] = df['Weight'].agg(lambda x: np.log(x + 1) if x > 0 else np.nan)
